# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [ ]:
# Import your libraries:

%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Challenge 1 - Import and Describe the Dataset

In this lab, we will use a dataset containing information about customer preferences. We will look at how much each customer spends in a year on each subcategory in the grocery store and try to find similarities using clustering.

The origin of the dataset is [here](https://archive.ics.uci.edu/ml/datasets/wholesale+customers).

In [ ]:
df = pd.read_csv('../data/Wholesale customers data.csv')
df.head()

#### Explore the dataset with mathematical and visualization techniques. What do you find?

Checklist:

* What does each column mean?
* Any categorical data to convert?
* Any missing data to remove?
* Column collinearity - any high correlations?
* Descriptive statistics - any outliers to remove?
* Column-wise data distribution - is the distribution skewed?
* Etc.

Additional info: Over a century ago, an Italian economist named Vilfredo Pareto discovered that roughly 20% of the customers account for 80% of the typical retail sales. This is called the [Pareto principle](https://en.wikipedia.org/wiki/Pareto_principle). Check if this dataset displays this characteristic.

In [ ]:
df.info()

In [ ]:
df.corr()

In [ ]:
df.describe()

In [ ]:
df.Region.value_counts(normalize=True)

In [ ]:
df.Channel.value_counts(normalize=True)

In [ ]:
fig, axs=plt.subplots(2,3, figsize=(17,8))
category = df.iloc[:,2:]

for i in range(category.shape[1]):
    axs[i//3,i%3].hist(category.iloc[:,i], bins=100)
    axs[i//3,i%3].set_title(category.columns[i])

plt.show()

In [ ]:
"""
Comments:
- Each row corresponds to the annual spending of a customer for each category of product. 
Channels are Horeca (1) or Retail (2) and Region are Lisbon (1), Oporto (2) and Other Region(3).
- No missing values
- We have categorical data that should be transformed as dummies: Channel and Region
- There are high correlations between Grocery and Detergents_Paper, also between Grocery and Milk
- The dataset is not really balanced because we have 70% rows for Region 3. 
- Data is not normally distributed, all data is right-skewed.

"""

# Challenge 2 - Data Cleaning and Transformation

If your conclusion from the previous challenge is the data need cleaning/transformation, do it in the cells below. However, if your conclusion is the data need not be cleaned or transformed, feel free to skip this challenge. But if you do choose the latter, please provide rationale.

In [ ]:
df = df.drop('Grocery',axis=1)

In [ ]:
df_dummies = pd.get_dummies(df, columns=['Channel','Region'], drop_first=True)
print(df_dummies.shape)
df_dummies.head()

In [ ]:
"""
- I'm droping the column Grocery because it is highly correlated to Milk and Detergents_Paper
- We won't transform the distribution of the data because we assume it is not necessary for 
unsupervised learning algorithms.
- 
"""

# Challenge 3 - Data Preprocessing

One problem with the dataset is the value ranges are remarkably different across various categories (e.g. `Fresh` and `Grocery` compared to `Detergents_Paper` and `Delicassen`). If you made this observation in the first challenge, you've done a great job! This means you not only completed the bonus questions in the previous Supervised Learning lab but also researched deep into [*feature scaling*](https://en.wikipedia.org/wiki/Feature_scaling). Keep on the good work!

Diverse value ranges in different features could cause issues in our clustering. The way to reduce the problem is through feature scaling. We'll use this technique again with this dataset.

#### We will use the `StandardScaler` from `sklearn.preprocessing` and scale our data. Read more about `StandardScaler` [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

*After scaling your data, assign the transformed data to a new variable `customers_scale`.*

In [ ]:
# Your import here:

from sklearn.preprocessing import StandardScaler

# Your code here:

scaler = StandardScaler()
scaler.fit(df_dummies)

customers_scale = scaler.transform(df_dummies)
pd.DataFrame(customers_scale, columns = df_dummies.columns)

# Challenge 3 - Data Clustering with K-Means

Now let's cluster the data with K-Means first. Initiate the K-Means model, then fit your scaled data. In the data returned from the `.fit` method, there is an attribute called `labels_` which is the cluster number assigned to each data record. What you can do is to assign these labels back to `customers` in a new column called `customers['labels']`. Then you'll see the cluster results of the original data.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
model = KMeans(3, random_state=0).fit(customers_scale)
df['labels'] = model.labels_

Count the values in `labels`.

In [ ]:
df.labels.value_counts()


# Challenge 4 - Data Clustering with DBSCAN

Now let's cluster the data using DBSCAN. Use `DBSCAN(eps=0.5)` to initiate the model, then fit your scaled data. In the data returned from the `.fit` method, assign the `labels_` back to `customers['labels_DBSCAN']`. Now your original data have two labels, one from K-Means and the other from DBSCAN.

In [ ]:
from sklearn.cluster import DBSCAN


In [ ]:
model_dbscan = DBSCAN(eps=0.5).fit(customers_scale)
df['labels_DBSCAN'] = (model_dbscan.labels_)

Count the values in `labels_DBSCAN`.

In [ ]:
df.labels_DBSCAN.value_counts()

In [ ]:
df.head()

# Challenge 5 - Compare K-Means with DBSCAN

Now we want to visually compare how K-Means and DBSCAN have clustered our data. We will create scatter plots for several columns. For each of the following column pairs, plot a scatter plot using `labels` and another using `labels_DBSCAN`. Put them side by side to compare. Which clustering algorithm makes better sense?

Columns to visualize:

* `Detergents_Paper` as X and `Milk` as y
* `Grocery` as X and `Fresh` as y
* `Frozen` as X and `Delicassen` as y

Visualize `Detergents_Paper` as X and `Milk` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
import seaborn as sns
sns.set()

sns.lmplot( x="Detergents_Paper", y="Milk", data=df, fit_reg=False, hue='labels')
sns.lmplot( x="Detergents_Paper", y="Milk", data=df, fit_reg=False, hue='labels_DBSCAN')
plt.show()

Visualize `Grocery` as X and `Fresh` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
sns.lmplot( x="Detergents_Paper", y="Fresh", data=df, fit_reg=False, hue='labels')
sns.lmplot( x="Detergents_Paper", y="Fresh", data=df, fit_reg=False, hue='labels_DBSCAN')
plt.show()


Visualize `Frozen` as X and `Delicassen` as y by `labels` and `labels_DBSCAN` respectively

In [ ]:
sns.lmplot( x="Frozen", y="Delicassen", data=df, fit_reg=False, hue='labels')
sns.lmplot( x="Frozen", y="Delicassen", data=df, fit_reg=False, hue='labels_DBSCAN')
plt.show()

Let's use a groupby to see how the mean differs between the groups. Group `customers` by `labels` and `labels_DBSCAN` respectively and compute the means for all columns.

In [ ]:
df.groupby(['labels','labels_DBSCAN']).agg('mean')


Which algorithm appears to perform better?

In [ ]:
"""
The second model seems to be more precise but it has too many clusters which makes it hard to read groups. 

"""

# Bonus Challenge 2 - Changing K-Means Number of Clusters

As we mentioned earlier, we don't need to worry about the number of clusters with DBSCAN because it automatically decides that based on the parameters we send to it. But with K-Means, we have to supply the `n_clusters` param (if you don't supply `n_clusters`, the algorithm will use `8` by default). You need to know that the optimal number of clusters differs case by case based on the dataset. K-Means can perform badly if the wrong number of clusters is used.

In advanced machine learning, data scientists try different numbers of clusters and evaluate the results with statistical measures (read [here](https://en.wikipedia.org/wiki/Cluster_analysis#External_evaluation)). We are not using statistical measures today but we'll use our eyes instead. In the cells below, experiment with different number of clusters and visualize with scatter plots. What number of clusters seems to work best for K-Means?

In [ ]:
model_2 = KMeans(5, random_state=0).fit(customers_scale)
df['labels_2'] = model_2.labels_

model_3 = KMeans(8, random_state=0).fit(customers_scale)
df['labels_3'] = model_3.labels_

sns.lmplot( x="Detergents_Paper", y="Fresh", data=df, fit_reg=False, hue='labels')
sns.lmplot( x="Detergents_Paper", y="Fresh", data=df, fit_reg=False, hue='labels_2')
sns.lmplot( x="Detergents_Paper", y="Fresh", data=df, fit_reg=False, hue='labels_3')
plt.show()

In [ ]:
"""
3 clusters seems to work better otherwise the number of cluster isn't significative because not so different 
from the others. 
"""

# Bonus Challenge 3 - Changing DBSCAN `eps` and `min_samples`

Experiment changing the `eps` and `min_samples` params for DBSCAN. See how the results differ with scatter plot visualization.

In [ ]:
model_dbscan_2 = DBSCAN(eps=3, min_samples=2).fit(customers_scale)
df['labels_DBSCAN_2'] = (model_dbscan_2.labels_)

model_dbscan_3 = DBSCAN(eps=1.5, min_samples=2).fit(customers_scale)
df['labels_DBSCAN_3'] = (model_dbscan_3.labels_)

model_dbscan_4 = DBSCAN(eps=3, min_samples=4).fit(customers_scale)
df['labels_DBSCAN_4'] = (model_dbscan_4.labels_)

sns.lmplot( x="Detergents_Paper", y="Milk", data=df, fit_reg=False, hue='labels_DBSCAN')
sns.lmplot( x="Detergents_Paper", y="Milk", data=df, fit_reg=False, hue='labels_DBSCAN_2')
sns.lmplot( x="Detergents_Paper", y="Milk", data=df, fit_reg=False, hue='labels_DBSCAN_3')
sns.lmplot( x="Detergents_Paper", y="Milk", data=df, fit_reg=False, hue='labels_DBSCAN_4')
plt.show()

In [ ]:
"""
labels_DBSCAN_4 seems to be more significant because all outliers are in the same cluster and we can see 
two distinct group between people who are most likely not buying Milk and Detergent and those who are 
actually buying more quantities. 

"""